In [1]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import window, column, desc, col 

- By default spark outputs 200 shuffle partitions

In [2]:

# spark = SparkSession.builder.appName("spark_definitive_guide").config("spark.driver.host", "localhost").getOrCreate()
spark = SparkSession.builder.appName("spark_definitive_guide").config("spark.driver.host", "localhost").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")

23/04/25 21:30:27 WARN Utils: Your hostname, AS-MAC-0006.local resolves to a loopback address: 127.0.0.1; using fd01:db8:1111:0:0:0:0:3 instead (on interface lo0)
23/04/25 21:30:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 21:30:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/25 21:30:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/25 21:30:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark.stop()

In [4]:
flight_data = "./../src/flight-data/csv/2015-summary.csv"
flight_data_2015 = spark.read.option("inferSchema", "true").option("header", "true").csv(flight_data)

In [11]:
flight_data_2015.sort("count").explain()
# flight_data_2015.sort("count").take(2)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#19 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#19 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [plan_id=55]
      +- FileScan csv [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/abhishekpathak/Code & Stuff/self/Learning/SPARK/spark_lear..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [6]:
path = "./../src/retail-data/by-day/*"
df = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load(path)

In [12]:
# df.printSchema()
df.createOrReplaceTempView("retail_data")
staticSchema = df.schema

In [20]:
df1 = df.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")
# df1.show()
df1 = df1.groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day")).sum("total_cost")
df1.sort(desc("CustomerId")).show(truncate=False)

+----------+------------------------------------------+------------------+
|CustomerId|window                                    |sum(total_cost)   |
+----------+------------------------------------------+------------------+
|18287.0   |{2011-10-28 05:30:00, 2011-10-29 05:30:00}|70.67999999999999 |
|18287.0   |{2011-05-22 05:30:00, 2011-05-23 05:30:00}|765.28            |
|18287.0   |{2011-10-12 05:30:00, 2011-10-13 05:30:00}|1001.3199999999999|
|18283.0   |{2011-11-10 05:30:00, 2011-11-11 05:30:00}|114.30000000000003|
|18283.0   |{2011-10-27 05:30:00, 2011-10-28 05:30:00}|114.65            |
|18283.0   |{2011-07-14 05:30:00, 2011-07-15 05:30:00}|143.19000000000008|
|18283.0   |{2011-11-23 05:30:00, 2011-11-24 05:30:00}|313.6499999999997 |
|18283.0   |{2011-06-14 05:30:00, 2011-06-15 05:30:00}|103.71999999999998|
|18283.0   |{2011-12-06 05:30:00, 2011-12-07 05:30:00}|208.00000000000009|
|18283.0   |{2011-04-21 05:30:00, 2011-04-22 05:30:00}|117.67999999999994|
|18283.0   |{2011-02-28 0

In [ ]:
streamingDF = spark.readStream.schema(staticSchema).option("maxFilesPerTrigger", 1).format("csv").option("header", "true").load(path)

purchaseByCustomerPerHour = streamingDF.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate").groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day")).sum("total_cost")

In [3]:
def longestCommonPrefix(strs):
        
        s = sorted(strs, key=len)
        if len(s) == 1 or len(s[0]) == 1:
            return s[0]
        print(s)
        comm = ""
        for i in range(len(s[0]) - 1):
            print(i)
            lett = s[0][i]
            print(lett)
            for j in s[1:]:
                if j[i] != lett:
                    print(j[i])
                    return comm
            comm = comm + lett
        return comm

print(longestCommonPrefix(["ab", "a"]))

a
